# GENERATE LABELS FOR SPLIT TRAINING DATA

In [22]:
#imports
import csv
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from pandas import Series, DataFrame
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, cluster

In [23]:
#convert string to nparray
def str_to_nplist(string):
    string = string.replace("[", "")
    string = string.replace("]", "")
    string = string.split(',')
    return np.asarray(string).astype(np.float32)

In [24]:
#converting labels to list
def convertlabelsfromString(s, ind):
    if type(s) == str:
        s = s.split(" ")
        s = [int(x) for x in s]
        return s
    else:
        return []

## Load Mean Features from HOG and CNN 

In [25]:
#reading csv files
train_data_hog = pd.read_csv("../train_data/bizfeatures_meanfeature.csv")
train_data_cnn = pd.read_csv("../train_data/bizfeatures_meanfeature_CNN.csv")
train_label_df = pd.read_csv('../train.csv')

In [26]:
train_data_cnn

,business_id,feature
0,3034,"[5.836266040802002, 1.5250771045684814, 1.1752..."
1,2805,"[4.011114597320557, 1.8206619024276733, 1.2678..."
2,485,"[4.010666370391846, 1.5894607305526733, 1.1684..."
3,1783,"[4.168568134307861, 1.6702979803085327, 1.4907..."
4,35,"[3.6683359146118164, 2.228299379348755, 1.5570..."
5,1313,"[4.841319561004639, 1.3613338470458984, 1.9253..."
6,2787,"[4.257303714752197, 1.9181667566299438, 1.6140..."
7,3216,"[3.5591235160827637, 1.3919190168380737, 1.580..."
8,3385,"[2.9830636978149414, 2.163705825805664, 1.0030..."
9,2896,"[4.059445858001709, 1.5130106210708618, 1.9239..."


## Create a new feature vector using HOG and CNN

In [27]:
train_data = pd.DataFrame({'business_id': [], 'feature': []})

In [28]:
train_data['business_id'] = train_data_hog['business_id']
train_data['feature'] = Series(train_data_hog['feature']).str.cat(train_data_cnn['feature'], sep=',')

In [29]:
str_to_nplist(train_data['feature'][0])

array([ 0.14311211,  0.07906874,  0.12857224, ...,  1.4460386 ,
        0.09451302,  1.69940174], dtype=float32)

In [30]:
train_data = train_data.sort_values(by='business_id')
train_label_df = train_label_df.sort_values(by='business_id')

In [31]:
mean_features = np.asarray([str_to_nplist(i) for i in train_data['feature']])
labels = train_label_df['labels'].values

## Using dimensionality reduction

In [32]:
print(mean_features.shape)
pca = PCA(n_components=2048)
X_new = pca.fit_transform(mean_features) 
print(X_new.shape)

(2000, 4096)
(2000, 2000)


In [47]:
#randomly breaking the dataset into 80:20 ratio for training and testing
msk = np.random.rand(len(mean_features)) < 0.8
mean_features_train = X_new[msk]
mean_features_test = X_new[~msk]
label_train_new_model = labels[msk]
label_test_new_model = labels[~msk]

In [48]:
X = mean_features_train
Y = label_train_new_model
label_binarizer = MultiLabelBinarizer()

In [49]:
Y = [convertlabelsfromString(Y[i],i) for i in range(Y.shape[0])]
Y = label_binarizer.fit_transform(Y)

In [50]:
classifier = OneVsRestClassifier(svm.SVC(kernel='linear'))
classifier.fit(X, Y)
prediction = classifier.predict(mean_features_test)

In [51]:
values = label_binarizer.inverse_transform(prediction)
inv_values = label_binarizer.fit_transform(values)

In [52]:
Y1 = [convertlabelsfromString(label_test_new_model[i],i) for i in range(label_test_new_model.shape[0])]
Y1 = label_binarizer.fit_transform(Y1)

In [53]:
f1_score(Y1,inv_values,average='micro')

0.78279055870929204

## GENERATE LABELS FOR TEST DATA

In [ ]:
test_data = pd.DataFrame({'business_id': [], 'feature': []})
test_data_hog = pd.read_csv("../test_data/bizfeaturestest_meanfeature.csv")
test_data_cnn = pd.read_csv("../test_data/bizfeatures_meanfeature_CNN_test.csv")

In [ ]:
test_data['business_id'] = test_data_hog['business_id']
test_data['feature'] = Series(test_data_hog['feature']).str.cat(test_data_cnn['feature'], sep=',')

In [199]:
mean_features_train = np.asarray([str_to_nplist(i) for i in train_data['feature']])
labels_train = train_label_df['labels'].values
mean_features_test = np.asarray([str_to_nplist(i) for i in test_data['feature']])

### Use the model to generate labels

In [202]:
X = mean_features
l_X = len(X)
X_test = test_mean_data
l_X_test = len(X_test)
print(X_test.shape)
X_new = np.vstack(X,X_test)
X_test_pca = pca.fit_transform(X_new)
Y = labels_train
label_binarizer = MultiLabelBinarizer()
print(X_test_pca.shape)
X_pca = X_test[:l_X][:]
X_test_pca = X_test[l_X+1:][:]

(2000, 2048)
(2000, 2000)


In [ ]:
Y = [convertlabelsfromString(Y[i],i) for i in range(Y.shape[0])]
Y = label_binarizer.fit_transform(Y)

In [ ]:
classifier = OneVsRestClassifier(svm.SVC(kernel='linear'))
classifier.fit(X_pca, Y)
prediction = classifier.predict(X_test_pca)

In [ ]:
values = label_binarizer.inverse_transform(prediction)
inv_values = label_binarizer.fit_transform(values)